##### What is prompt template?

A Prompt Template is a predefined structure for prompts where only some parts change.

##### Why do we use prompt templates?
Instead of writing prompts again and again, we create a template and fill values dynamically

##### Why Prompt Templates Matter (Before We Go Deeper)

Without templates:
+ prompts change every time
+ outputs become unpredictable
+ hallucinations increase
+ agents break

With templates:
+ consistent instructions
+ controlled outputs
+ reusable prompts
+ safer AI behavior

##### Basic Prompt Template
A simple text prompt with placeholders. Think of it like: "Fill in the blanks"

In [2]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="Explain {topic} in simple words",
    input_variables=["topic"]
)

prompt.format(topic="LLM")


'Explain LLM in simple words'

Structured Output Prompt

Problem Without Structure you face below issues: 
+ Sometimes paragraph
+ Sometimes bullet points
+ Sometimes random format

This is hallucination in format, not facts.
To resolve this problem: We tell the model exactly how to respond.

In [4]:
prompt = """
Explain {topic}.
Return output strictly in JSON format:

{{
  "definition": "",
  "example": "",
  "use_case": ""
}}
"""


Why This Is Important

✔ Predictable output
✔ Easy to parse
✔ Required for APIs & agents

📌 Agents REQUIRE structured outputs
***

##### ChatPromptTemplate

Its used for chat-based models such as GPT, Claude, etc.

Supports roles:
+ system
+ human
+ ai

Why This Is Powerful
+ System message controls behavior
+ Human message controls task
+ Separates instruction vs question

Where to Use

1. Chatbots
2. Agents
3. RAG systems

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful teacher"),
    ("human", "Explain {topic} to a beginner")
])


##### Few-Shot Prompts: You show examples so the model learns the pattern.

Why Few-Shot Works?

LLMs are pattern learners. Examples reduce guessing.

When should you use Few-Shot

+ Classification
+ Formatting tasks
+ Data extraction
+ Reducing hallucinations

In [9]:
prompt = """
Q: What is AI?
A: AI is the ability of machines to think.

Q: What is ML?
A: ML is a way machines learn from data.

Q: What is LLM?
A:
"""


##### Why Prompt Templates Reduce Hallucinations

Here are the reasons why prompt templates reduces hallucinations
1. Clear Instructions: ❌ “Explain AI” ✅ “Explain AI in 3 bullet points for beginners”
2. Fixed Structure: Templates lock the response shape { "answer": "", "example": "" }
3. Consistency Across Calls: Same prompt structure → similar outputs. This is critical for agents
4. Separation of Roles: System ≠ Human ≠ Output. Model understands who says what
5. Reusability & Control: You don’t "re-prompt" every time: One correct template >>>>> reused everywhere
   
Prompt templates reduce hallucinations by enforcing clear instructions, fixed structure, consistent behavior, and example-based guidance.

In [11]:
# pip install -U langchain langchain-openai pydantic python-dotenv

import os
from dotenv import load_dotenv

from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

In [12]:
class ProductInfo(BaseModel):
    name: str = Field(..., description="Product name")
    price_inr: int = Field(..., ge=0, description="Price in INR (non-negative integer)")
    category: str = Field(..., description="Category like saree/kurti/coords")
    sizes: list[str] = Field(..., description="Available sizes, e.g., ['M','L','XL']")
    in_stock: bool = Field(..., description="Availability")

# All flights to London at 29999

# class FlightInfo(BaseModel):
#     destination: str = Field(..., description="Travel destination")
#     price_inr: int = Field(..., ge=0, description="Prices in INR (non-negative integer)")

In [13]:
load_dotenv(override=True)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

parser = PydanticOutputParser(pydantic_object=ProductInfo)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a strict data extraction assistant. Output must follow the schema."),
    ("human",
     "Extract product details from this text:\n\n"
     "{text}\n\n"
     "{format_instructions}")
])

chain = prompt | llm | parser


In [14]:
text = """
New Arrival
Maslin kurti set with handwork, Dupatta: digital print.
Price Rs 1799. Sizes M to XXL. Available now.
"""

result = chain.invoke({
    "text": text,
    "format_instructions": parser.get_format_instructions()
})

result

ProductInfo(name='Maslin kurti set with handwork', price_inr=1799, category='kurti', sizes=['M', 'L', 'XL', 'XXL'], in_stock=True)

In [15]:
print(result.name)
print(result.price_inr)
print(result.sizes)
print(result.model_dump())

Maslin kurti set with handwork
1799
['M', 'L', 'XL', 'XXL']
{'name': 'Maslin kurti set with handwork', 'price_inr': 1799, 'category': 'kurti', 'sizes': ['M', 'L', 'XL', 'XXL'], 'in_stock': True}


In [16]:
bad_text = "Name: Saree, Price: 'five hundred' INR, Category: saree, Sizes: M, In stock: yes"

try:
    chain_strict.invoke({
        "text": bad_text,
        "format_instructions": parser.get_format_instructions()
    })
except Exception as e:
    print("Validation failed ✅")
    print(type(e))
    print(e)

Validation failed ✅
<class 'NameError'>
name 'chain_strict' is not defined


In [17]:
bad_text = "Price is -500 INR, size: M, product: Saree, in stock: yes"

strict_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a strict data extraction assistant.\n"
     "IMPORTANT: Do not correct, transform, or sanitize values.\n"
     "If the text says price is -500, output -500.\n"
     "Output must follow the schema."),
    ("human",
     "Extract product details from this text:\n\n"
     "{text}\n\n"
     "{format_instructions}")
])

chain_strict = strict_prompt | llm | parser

try:
    chain_strict.invoke({
        "text": bad_text,
        "format_instructions": parser.get_format_instructions()
    })
except Exception as e:
    print("Validation failed ✅")
    print(type(e))
    print(e)


Validation failed ✅
<class 'langchain_core.exceptions.OutputParserException'>
Failed to parse ProductInfo from completion {"name": "Saree", "price_inr": -500, "category": "saree", "sizes": ["M"], "in_stock": true}. Got: 1 validation error for ProductInfo
price_inr
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-500, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than_equal
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 


#### Pydantic is the “data contract”, and structured output makes the LLM follow that contract so our code can safely use the result.